### Introduction

**Toytree** is a Python tree plotting library designed for use inside 
[jupyter notebooks](http://jupyter.org). In fact, this tutorial was created in a Jupyter notebook and assumes that you are following-along in a notebook of your own. Before getting started with **Toytree** it may be helpful to read some background on [**Toyplot**](http://toyplot.rtfd.io) to understand how figures are generated and displayed. If you aren’t using a notebook, you should read Toyplot's user guide section on [rendering](http://toyplot.readthedocs.io/en/stable/rendering.html#rendering) for some important information on how to display your figures.


To begin, import toyplot, toytree, and numpy. 

In [1]:
import toytree     ## a tree plotting library
import toyplot     ## a general plotting library
import numpy as np ## data generation 

In [2]:
print(toytree.__version__)
print(toyplot.__version__)

0.1.5
0.16.1


### Toytree Class objects

Toytree has only two Python Class objects, the 
<span style="text-decoration:underline;">Toytree</span> object
that is used to represent a single tree, and the 
<span style="text-decoration:underline;">Multitree</span> object
that is used to represent a list of trees. 

To create a Toytree instance you must load in a tree representation from a newick string or file using the tree parsing function `toytree.tree`. This will parse the newick data to return a Toytree Object with the tree structure stored in memory. To do this, Toytree uses a stripped-down version of the [ete](http://etetoolkit.org) module (which we call ete3mini) for newick parsing and tree representation. Therefore, nearly all of the machinery that is available in [ete](http://etetoolkit.org) to modify and traverse trees is also available to Toytree objects. See the [Tree traversal/modification](toytrees.md) section for more details. 

### Reading/Parsing trees
Below are two trees in newick format. The first has edge lengths and support values, the second has edge-lengths and node-labels. These are two different ways of writing tree data in a serialized format. To parse either format you must tell toytree the format of the newick string following the [tree parsing formats in ete](http://etetoolkit.org/docs/latest/tutorial/tutorial_trees.html#reading-and-writing-newick-trees). The most commonly used format is 0, which is also the default. Toytree can also parse extended newick format files in which many types of metadata can be appended to a single tree.  

In [3]:
## newick with edge-lengths & support values
newick = "((a:1,b:1)90:2,(c:3,(d:1, e:1)100:2)100:1)100;"
tre0 = toytree.tree(newick, format=0)

## newick with edge-lengths & string node-labels
newick = "((a:1,b:1)A:2,(c:3,(d:1, e:1)B:2)C:1)root;"
tre1 = toytree.tree(newick, format=1)

### Drawing trees (basics)

In [4]:
## draw one tree
tre0.draw();

e d c b a

In [5]:
## draw multiple trees
tre0.draw();
tre1.draw();

e d c b a

e d c b a

In [6]:
## organize trees on a canvas (more on this later)
canvas = toyplot.Canvas(width=400, height=200)
ax0 = canvas.cartesian(bounds=('10%', '40%', '10%', '90%'))
ax1 = canvas.cartesian(bounds=('60%', '90%', '10%', '90%'))
tre0.draw(axes=ax0);
tre1.draw(axes=ax1);
ax0.show=False
ax1.show=False

e d c b a e d c b a

### The Canvas 
When you call the `toytree.draw()` function it returns two Toyplot objects which are used to display the figure. The first is the Canvas, which is the HTML element that holds the figure, and the second is a Cartesian axes object, which represent the coordinates for the plot. You can catch these objects when they are returned by the `draw()` function to further manipulate the plot. 

In [7]:
## catch canvas and axes from draw()
canvas, axes = tre0.draw(width=200, height=250, padding=25)

## e.g., turn on or off some axes elements
axes.show = True
axes.y.show = False
axes.x.ticks.show = True

e d c b a -4 -3 -2 -1 0 1

Or, instead of catching the canvas and axes auto-generated by the `toytree.draw()` function you can instead generate the canvas and axes yourself using Toyplot and pass the axes object as an argument to `draw()` to embed the tree within the axes coordinates. This is a useful way to combine multiple figures on a single canvas, or to annotate axes.

In [8]:
## create the canvas 
canvas = toyplot.Canvas(width=250, height=250)
axes = canvas.cartesian(margin=50)

## add tree to existing canvas
canvas, axes = tre0.draw(axes=axes)

## further modify the axes
axes.y.show = False
axes.x.ticks.show = True
axes.x.label.text = "Divergence time (Ma)"

e d c b a -4 -2 0 Divergence time (Ma)

### Drawing trees (advanced)
See the sections on [node-labels](node-labels.md) and [tip-labels](tip-labels.md) for detailed instructions on how to modify these features. Here I will focus on how Toytree helps to ensure that users display the proper data on the tree to avoid mistakes. Toytree provides the *magic command* `node_labels=True`, which embeds interactive features into the plot so that you can hover over nodes with your cursor and can see all of the information that is available for that node extracted from the tree.  

In [9]:
## 'False' or None (default) blocks node labels
canvas, axes = tre0.draw(
    node_labels=None,
    )

## But, if you set a size then interactive hover nodes are still drawn
canvas, axes = tre0.draw(
    node_labels=None,
    node_size=12,
    node_color='grey'
    )

## trees have a set of default features that can be accessed like 'support'
canvas, axes = tre1.draw(
    node_labels="name",
    node_size=18
    )

## or a list of values, including values that can be retrieved from the tree obj
canvas, axes = tre1.draw(
    node_labels=tre0.get_node_values("support", show_root=0, show_tips=0),
    node_size=18
    )

e d c b a

e d c b a idx: 0
name: e
dist: 1
support: 100
height: 0 idx: 1
name: d
dist: 1
support: 100
height: 0 idx: 2
name: c
dist: 3
support: 100
height: 0 idx: 3
name: b
dist: 1
support: 100
height: 1 idx: 4
name: a
dist: 1
support: 100
height: 1 idx: 5
name: i5
dist: 2
support: 100
height: 1 idx: 6
name: i6
dist: 1
support: 100
height: 3 idx: 7
name: i7
dist: 2
support: 90
height: 2 idx: 8
name: i8
dist: 0
support: 100
height: 4

e d c b a idx: 0
name: e
dist: 1
support: 100
height: 0 e idx: 1
name: d
dist: 1
support: 100
height: 0 d idx: 2
name: c
dist: 3
support: 100
height: 0 c idx: 3
name: b
dist: 1
support: 100
height: 1 b idx: 4
name: a
dist: 1
support: 100
height: 1 a idx: 5
name: B
dist: 2
support: 100
height: 1 B idx: 6
name: C
dist: 1
support: 100
height: 3 C idx: 7
name: A
dist: 2
support: 100
height: 2 A idx: 8
name: root
dist: 0
support: 100
height: 4 root

e d c b a idx: 5
name: B
dist: 2
support: 100
height: 1 100 idx: 6
name: C
dist: 1
support: 100
height: 3 100 idx: 7
name: A
dist: 2
support: 100
height: 2 90

### Extracting data from the tree

Although you *can* enter values for the node_labels or tip_labels directly into the draw() function as a list, doing so is frowned upon because it can often lead to errors if the values are entered in the incorrect order, or if the tree is re-oriented, ladderized, or pruned. Instead, Toytree aims to encourage users to *always* extract the data directly from the Tree object itself, such that the data will always be in sync with the tree. The interactive feature `node_label=True` is one example of this, where all of the information for each node is shown, and extracted from the tree, so you know for sure that the data are in sync. 

In addition, we provide convenience functions to extract data from a Toytree object in the order that it will be plotted on the tree. For node values the function `get_node_values()` should be used, and for tip labels the function `get_tip_labels()` should be used. Below we show some example usage of `get_node_values()`. See the section on [node-labels](node-labels.md) and [modifying the tree object](modifying-the-tree.md) for more information. 

In [10]:
## get node values returns a list of values, empty by default
tre0.get_node_values()

['', '', '', '', '', ' ', ' ', ' ', '']

In [11]:
## it takes up to three arguments
tre0.get_node_values(feature='idx', show_root=False, show_tips=False)

['', '', '', '', '', 5, 6, 7, '']

In [12]:
## it can access any feature in the tree 
tre0.get_node_values(feature='name', show_root=False, show_tips=False)

['', '', '', '', '', 'i5', 'i6', 'i7', '']

In [13]:
## and either hide or show the root & tip values
tre0.get_node_values(feature='name', show_root=True, show_tips=True)

['e', 'd', 'c', 'b', 'a', 'i5', 'i6', 'i7', 'i8']

In [14]:
## show the index number (idx) of each node
tre0.draw(node_labels=tre0.get_node_values("idx", True, True));

e d c b a idx: 0
name: e
dist: 1
support: 100
height: 0 0 idx: 1
name: d
dist: 1
support: 100
height: 0 1 idx: 2
name: c
dist: 3
support: 100
height: 0 2 idx: 3
name: b
dist: 1
support: 100
height: 1 3 idx: 4
name: a
dist: 1
support: 100
height: 1 4 idx: 5
name: i5
dist: 2
support: 100
height: 1 5 idx: 6
name: i6
dist: 1
support: 100
height: 3 6 idx: 7
name: i7
dist: 2
support: 90
height: 2 7 idx: 8
name: i8
dist: 0
support: 100
height: 4 8

In [15]:
## show node names
tre0.draw(node_labels=tre0.get_node_values("name", True, True));

e d c b a idx: 0
name: e
dist: 1
support: 100
height: 0 e idx: 1
name: d
dist: 1
support: 100
height: 0 d idx: 2
name: c
dist: 3
support: 100
height: 0 c idx: 3
name: b
dist: 1
support: 100
height: 1 b idx: 4
name: a
dist: 1
support: 100
height: 1 a idx: 5
name: i5
dist: 2
support: 100
height: 1 i5 idx: 6
name: i6
dist: 1
support: 100
height: 3 i6 idx: 7
name: i7
dist: 2
support: 90
height: 2 i7 idx: 8
name: i8
dist: 0
support: 100
height: 4 i8

In [16]:
## show support values (hides root & tip values by default)
tre0.draw(node_labels=tre0.get_node_values("support"));

## alternatively, if you only enter the keyword of a feature both roots and tips are shown
tre0.draw(node_labels='support');

e d c b a idx: 5
name: i5
dist: 2
support: 100
height: 1 100 idx: 6
name: i6
dist: 1
support: 100
height: 3 100 idx: 7
name: i7
dist: 2
support: 90
height: 2 90

e d c b a idx: 0
name: e
dist: 1
support: 100
height: 0 100 idx: 1
name: d
dist: 1
support: 100
height: 0 100 idx: 2
name: c
dist: 3
support: 100
height: 0 100 idx: 3
name: b
dist: 1
support: 100
height: 1 100 idx: 4
name: a
dist: 1
support: 100
height: 1 100 idx: 5
name: i5
dist: 2
support: 100
height: 1 100 idx: 6
name: i6
dist: 1
support: 100
height: 3 100 idx: 7
name: i7
dist: 2
support: 90
height: 2 90 idx: 8
name: i8
dist: 0
support: 100
height: 4 100

In [17]:
## parse the info and plot as node colors
colors = [tre0.colors[0] if i==100 else tre0.colors[1] \
          for i in tre0.get_node_values("support", True, True)]

## plot node colors
tre0.draw(
    node_labels=False, 
    node_color=colors,
    node_size=15,
);

e d c b a idx: 0
name: e
dist: 1
support: 100
height: 0 idx: 1
name: d
dist: 1
support: 100
height: 0 idx: 2
name: c
dist: 3
support: 100
height: 0 idx: 3
name: b
dist: 1
support: 100
height: 1 idx: 4
name: a
dist: 1
support: 100
height: 1 idx: 5
name: i5
dist: 2
support: 100
height: 1 idx: 6
name: i6
dist: 1
support: 100
height: 3 idx: 7
name: i7
dist: 2
support: 90
height: 2 idx: 8
name: i8
dist: 0
support: 100
height: 4

### Edge lengths

In [18]:
## plot w/o using edge lengths
tre0.draw(use_edge_lengths=False);

## use edge lengths but align tips
tre0.draw(tip_labels_align=True);


e d c b a

e d c b a

In [19]:
### rotate the tree
tre0.draw(tip_labels_align=True, orient='down');


e d c b a